In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgbm
import xgboost
import multiprocessing
import optuna

from sklearn.neighbors import KNeighborsRegressor

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [12]:
num_cores = multiprocessing.cpu_count()
num_cores

4

In [21]:
X_train = pd.read_csv("./Dataset/train_final.csv")
X_train.drop("Unnamed: 0", axis = 1, inplace=True)
X_train.head()

,Count,Open,Volume,Asset_ID,Target,Range_Close_Open,Range_High_Low
0,-0.420178,-0.231602,-0.163735,0,0.000148,0.009311,-0.106561
1,-0.406696,-0.231558,-0.163475,0,0.000393,0.009823,-0.106239
2,-0.412850,-0.231527,-0.163609,0,0.000549,0.009775,-0.106443
3,-0.411005,-0.231471,-0.163602,0,0.000006,0.010801,-0.105861
4,-0.389464,-0.230984,-0.162833,0,0.005618,0.011829,-0.102587


In [ ]:
%%time

from sklearn.model_selection import cross_validate

def cross_validate_manual(X, y, model):
    cv = cross_validate(
        estimator = model,
        X = X,
        y = y,
        scoring = ["neg_mean_squared_error"],
        cv = 5,
    )
    
    return cv["test_neg_mean_squared_error"].mean()

In [22]:
model_lgbm = LGBMRegressor()
mean_lgbm = cross_validate_manual(X_train, y_train, model_lgbm)
print("LGBM: ", np.abs(10e6*mean_lgbm))

LGBM:  0.12017209237859969


In [23]:
model_lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [24]:
import optuna.integration.lightgbm as lgb
def objective(trial):
    param_grid = {
        'num_jobs': num_cores - 1,
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 0.3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
    
    model_lgbm = LGBMRegressor()
    mean_lgbm = cross_validate_manual(X_train, y_train, model_lgbm)
#     print("LGBM: ", np.abs(10e6*mean_lgbm))

    return np.abs(10e6*mean_lgbm)

In [25]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2022-04-24 17:51:21,399] A new study created in memory with name: no-name-39582fc4-0af4-439a-9a7a-e487abfc2f03
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:51:22,248] Trial 0 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.10545152476290759, 'num_leaves': 1600, 'max_depth': 6, 'min_data_in_leaf': 7000, 'max_bin': 247, 'lambda_l1': 95, 'lambda_l2': 50, 'min_gain_to_split': 7.052216867935241, 'reg_alpha': 0.13302597513428918, 'reg_lambda': 0.5360701301180708, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:51:32,777] Trial 11 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.15071668058293236, 'num_leaves': 1440, 'max_depth': 5, 'min_data_in_leaf': 9500, 'max_bin': 245, 'lambda_l1': 60, 'lambda_l2': 55, 'min_gain_to_split': 9.429349907765777, 'reg_alpha': 0.15368293289359183, 'reg_lambda': 0.13735815839953947, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:51:33,880] Tria

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:51:43,714] Trial 22 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.06314372465973295, 'num_leaves': 1440, 'max_depth': 12, 'min_data_in_leaf': 7000, 'max_bin': 222, 'lambda_l1': 45, 'lambda_l2': 30, 'min_gain_to_split': 1.5934067988358445, 'reg_alpha': 0.18746142140915525, 'reg_lambda': 0.4077232188352091, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:51:44,559] Tri

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:51:54,310] Trial 33 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.010036661366265912, 'num_leaves': 2280, 'max_depth': 9, 'min_data_in_leaf': 7900, 'max_bin': 241, 'lambda_l1': 0, 'lambda_l2': 50, 'min_gain_to_split': 6.673232131198498, 'reg_alpha': 0.11864114847851306, 'reg_lambda': 0.3213007565619734, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:5

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:06,051] Trial 44 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.10630204557668119, 'num_leaves': 1960, 'max_depth': 6, 'min_data_in_leaf': 8600, 'max_bin': 200, 'lambda_l1': 5, 'lambda_l2': 5, 'min_gain_to_split': 4.37423942750213, 'reg_alpha': 0.041695729186274996, 'reg_lambda': 0.7456253145948615, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:17,385] Trial 55 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2178862827891455, 'num_leaves': 2060, 'max_depth': 10, 'min_data_in_leaf': 5900, 'max_bin': 233, 'lambda_l1': 0, 'lambda_l2': 45, 'min_gain_to_split': 6.824825738416589, 'reg_alpha': 0.16494184331326708, 'reg_lambda': 0.3127493782628253, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:28,613] Trial 66 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.09513534571158207, 'num_leaves': 580, 'max_depth': 10, 'min_data_in_leaf': 2800, 'max_bin': 259, 'lambda_l1': 95, 'lambda_l2': 25, 'min_gain_to_split': 12.874541144655762, 'reg_alpha': 0.14774375744858434, 'reg_lambda': 0.6560384973434845, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:40,009] Trial 77 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.19735341925757252, 'num_leaves': 2300, 'max_depth': 8, 'min_data_in_leaf': 7200, 'max_bin': 204, 'lambda_l1': 100, 'lambda_l2': 45, 'min_gain_to_split': 7.181335829537346, 'reg_alpha': 0.17906725027420395, 'reg_lambda': 0.1893117668923291, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:51,531] Trial 88 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.13317880389378273, 'num_leaves': 1360, 'max_depth': 3, 'min_data_in_leaf': 2400, 'max_bin': 274, 'lambda_l1': 60, 'lambda_l2': 60, 'min_gain_to_split': 12.270760823915248, 'reg_alpha': 0.0851409703098693, 'reg_lambda': 0.6256201802989783, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:52:52,570] Trial

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:53:03,605] Trial 99 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.28789999973303637, 'num_leaves': 640, 'max_depth': 10, 'min_data_in_leaf': 4300, 'max_bin': 253, 'lambda_l1': 40, 'lambda_l2': 15, 'min_gain_to_split': 0.16557104492410946, 'reg_alpha': 0.1014250811874546, 'reg_lambda': 0.7020725482278862, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.12017209237859969.


In [26]:
study.best_params

{'n_estimators': 10000,
 'learning_rate': 0.10545152476290759,
 'num_leaves': 1600,
 'max_depth': 6,
 'min_data_in_leaf': 7000,
 'max_bin': 247,
 'lambda_l1': 95,
 'lambda_l2': 50,
 'min_gain_to_split': 7.052216867935241,
 'reg_alpha': 0.13302597513428918,
 'reg_lambda': 0.5360701301180708,
 'bagging_fraction': 0.6000000000000001,
 'bagging_freq': 1,
 'feature_fraction': 0.2}

In [27]:
model_lgbm = LGBMRegressor(n_estimators=10000, learning_rate=0.11524, num_leaves=2380, max_depth=5, min_data_in_leaf= 600, lambda_l1=40, lambda_l2=20, min_gain_to_split=11.932, reg_alpha=0.2351, reg_lambda=0.2, baging_fraction= 0.7, bagging_freq=1, feature_fraction=0.6)
mean_lgbm = cross_validate_manual(X_train, y_train, model_lgbm)
print("LGBM: ", np.abs(10e6*mean_lgbm))

[LightGBM] [Warning] Unknown parameter: baging_fraction
[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.2351 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] min_gain_to_split is set=11.932, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.932
[LightGBM] [Warning] lambda_l2 is set=20, reg_lambda=0.2 will be ignored. Current value: lambda_l2=20
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=600
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] Unknown parameter: baging_fraction
[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.2351 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] min_gain_to_split is set=11.932, min_split_gain=0.0 will be ignored. Cu

In [28]:
def objective_xgb(trial):
    param_grid = {
        'n_jobs': num_cores - 1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 500),
        'max_leaves': trial.suggest_int('max_leaves', 1, 5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 0.3),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
    }

    model_xgb = XGBRegressor()
    mean_xgb = cross_validate_manual(X_train, y_train, model_xgb)
    
    return np.abs(10e6*mean_xgb)

In [29]:
study_xgb = optuna.create_study()
study_xgb.optimize(objective, n_trials=100)

[I 2022-04-24 17:53:48,577] A new study created in memory with name: no-name-2356631c-bb2b-4438-999a-0f1fea4938f4
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:53:49,412] Trial 0 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1361709887234184, 'num_leaves': 2960, 'max_depth': 8, 'min_data_in_leaf': 5400, 'max_bin': 222, 'lambda_l1': 30, 'lambda_l2': 80, 'min_gain_to_split': 8.034321470919213, 'reg_alpha': 0.28743895460283664, 'reg_lambda': 0.15386204858977315, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the ran

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:53:58,738] Trial 11 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.010573301755708031, 'num_leaves': 2340, 'max_depth': 6, 'min_data_in_leaf': 2700, 'max_bin': 214, 'lambda_l1': 65, 'lambda_l2': 70, 'min_gain_to_split': 6.4254569637088945, 'reg_alpha': 0.1365408635241615, 'reg_lambda': 0.33192883494045883, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:53:59,753] Tri

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:08,895] Trial 22 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.23121552863058253, 'num_leaves': 2700, 'max_depth': 9, 'min_data_in_leaf': 3800, 'max_bin': 232, 'lambda_l1': 90, 'lambda_l2': 65, 'min_gain_to_split': 12.194127930761793, 'reg_alpha': 0.29763824386014603, 'reg_lambda': 0.3693163325597154, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:09,765] Tria

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:20,906] Trial 33 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2923015693251561, 'num_leaves': 980, 'max_depth': 12, 'min_data_in_leaf': 4900, 'max_bin': 210, 'lambda_l1': 0, 'lambda_l2': 5, 'min_gain_to_split': 0.10125523548691728, 'reg_alpha': 0.2395223176875355, 'reg_lambda': 0.0880886434090961, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:22,266] Trial 3

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:33,897] Trial 44 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1768727108101985, 'num_leaves': 2980, 'max_depth': 8, 'min_data_in_leaf': 6000, 'max_bin': 284, 'lambda_l1': 80, 'lambda_l2': 30, 'min_gain_to_split': 2.43388842440266, 'reg_alpha': 0.20452833792235803, 'reg_lambda': 0.2998139508352617, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:48,534] Trial 55 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2920679619130091, 'num_leaves': 1060, 'max_depth': 12, 'min_data_in_leaf': 2700, 'max_bin': 220, 'lambda_l1': 5, 'lambda_l2': 5, 'min_gain_to_split': 0.5554918655194543, 'reg_alpha': 0.2430495575369908, 'reg_lambda': 0.0755242307729194, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:54:49,600] Trial 5

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:55:00,068] Trial 66 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.15684458575083943, 'num_leaves': 2880, 'max_depth': 6, 'min_data_in_leaf': 3900, 'max_bin': 244, 'lambda_l1': 100, 'lambda_l2': 60, 'min_gain_to_split': 14.042744168431406, 'reg_alpha': 0.08042993963830848, 'reg_lambda': 0.44741566341849814, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 1

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:55:13,281] Trial 77 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2701529290278014, 'num_leaves': 720, 'max_depth': 3, 'min_data_in_leaf': 900, 'max_bin': 213, 'lambda_l1': 10, 'lambda_l2': 10, 'min_gain_to_split': 1.1279644148912469, 'reg_alpha': 0.25128528425293317, 'reg_lambda': 0.12072316781098495, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:55:14,365] Trial 

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:55:25,755] Trial 88 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.040467225503266216, 'num_leaves': 600, 'max_depth': 6, 'min_data_in_leaf': 8000, 'max_bin': 240, 'lambda_l1': 65, 'lambda_l2': 90, 'min_gain_to_split': 11.658825008114786, 'reg_alpha': 0.05270007388682546, 'reg_lambda': 0.5831427433424453, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 0 with value: 0.12017209237859969.
/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17

/home/kavya/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-04-24 17:55:37,738] Trial 99 finished with value: 0.12017209237859969 and parameters: {'n_estimators': 10000, 'learning_rate': 0.18135938578864313, 'num_leaves': 2580, 'max_depth': 10, 'min_data_in_leaf': 6900, 'max_bin': 226, 'lambda_l1': 95, 'lambda_l2': 80, 'min_gain_to_split': 11.205060089233278, 'reg_alpha': 0.284362771460267, 'reg_lambda': 0.3060260823068432, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 0 with value: 0.12017209237859969.


In [30]:
study_xgb.best_params

{'n_estimators': 10000,
 'learning_rate': 0.1361709887234184,
 'num_leaves': 2960,
 'max_depth': 8,
 'min_data_in_leaf': 5400,
 'max_bin': 222,
 'lambda_l1': 30,
 'lambda_l2': 80,
 'min_gain_to_split': 8.034321470919213,
 'reg_alpha': 0.28743895460283664,
 'reg_lambda': 0.15386204858977315,
 'bagging_fraction': 0.5,
 'bagging_freq': 1,
 'feature_fraction': 0.7}

In [32]:
model_xgb = XGBRegressor(n_estimators=10000, learning_rate=0.06891, max_depth=9, max_bin=245, reg_alpha=0.09297, reg_lambda = 0.14678)
mean_xgb = cross_validate_manual(X_train, y_train, model_xgb)
np.abs(10e6*mean_xgb)

0.1343926745283607